In [1]:
import mediapipe as mp
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
import os
import cv2
import math
import numpy as np
import copy
import itertools
import csv
from collections import Counter
from collections import deque
from utils import CvFpsCalc
from matplotlib import pyplot as plt

In [2]:
DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480


def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  return img

def load_images_from_folder(folder):
    images = []
    
    for filename in os.listdir(folder):
        #print(filename)
        os.chdir(folder)
        img = cv2.imread(filename)
        #print(img)
        if img is not None:
            img=resize_and_show(img)
            images.append(img)
    return images

In [3]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [4]:
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))
    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [83]:
def logging_csv(number, mode, landmark_list, point_history_list):
# if mode == 0:
#     pass
# if mode == 1 and (0 <= number <= 9):
    csv_path = 'model/keypoint_classifier/keypoint_all-letters_right.csv'
    #csv_path = 'model/keypoint_classifier/keypoint_withoutQ&X.csv'

    # print(len(landmark_list))
    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f)
        # writer.writerow([number, *landmark_list])
        landmark_list.insert(0, number)
        writer.writerow(landmark_list)
        del landmark_list[0]

    return

In [7]:
def linear_approx_x(x_1,x_2):
    points_x = []
    if(x_1>=0 and x_2>=0):
        if(x_1 > x_2):
            points_x.append(x_2 + ((x_1 - x_2) * 2 /3))
            points_x.append(x_2 + ((x_1 - x_2) * 1 /3))
            return points_x
        else:
            points_x.append(x_1 + ((x_2 - x_1) * 1 /3))
            points_x.append(x_1 + ((x_2 - x_1) * 2 /3))
            return points_x

    if(x_1>=0 and x_2<=0):
        points_x.append(x_2 + ((x_1 + abs(x_2)) * 2 /3))
        points_x.append(x_2 + ((x_1 + abs(x_2)) * 1 /3))
        return points_x

    if(x_1<=0 and x_2>=0):
        points_x.append(x_1 + ((x_2 + abs(x_1)) * 1 /3))
        points_x.append(x_1 + ((x_2 + abs(x_1)) * 2 /3))
        return points_x
    
    if(x_1<=0 and x_2<=0):
        if(x_1 > x_2):
            points_x.append(x_1 + ((x_2)-(x_1)) * 1/3)
            points_x.append(x_1 + ((x_2)-(x_1)) * 2/3)
            return points_x
        else:
            points_x.append(x_1 + ((x_2)-(x_1)) * 1/3)
            points_x.append(x_1 + ((x_2)-(x_1)) * 2/3)
            return points_x
    


In [8]:
def linear_approx_y(y_1,y_2):
    points_y = []
    if(y_1>=0 and y_2>=0):
        if(y_1 > y_2):
            points_y.append(y_2 + ((y_1 - y_2) * 2 /3))
            points_y.append(y_2 + ((y_1 - y_2) * 1 /3))
            return points_y
        else:
            points_y.append(y_1 + ((y_2 - y_1) * 1 /3))
            points_y.append(x_1 + ((y_2 - y_1) * 2 /3))
            return points_y

    if(y_1>=0 and y_2<=0):
        points_y.append(y_2 + ((y_1 + abs(y_2)) * 2 /3))
        points_y.append(y_2 + ((y_1 + abs(y_2)) * 1 /3))
        return points_y

    if(y_1<=0 and y_2>=0):
        points_y.append(y_1 + ((y_2 + abs(y_1)) * 1 /3))
        points_y.append(y_1 + ((y_2 + abs(y_1)) * 2 /3))
        return points_y
    
    if(y_1<=0 and y_2<=0):
        if(y_1 > y_2):
            points_y.append(y_1 + ((y_2)-(y_1)) * 1/3)
            points_y.append(y_1 + ((y_2)-(y_1)) * 2/3)
            return points_y
        else:
            points_y.append(y_1 + ((y_2)-(y_1)) * 1/3)
            points_y.append(y_1 + ((y_2)-(y_1)) * 2/3)
            return points_y
    


In [10]:
# filling empty frames linearly
def fill_coordinates(relative_list_in_fnc):
    list_x=[0,0,0,0,0,0,0]
    list_y=[0,0,0,0,0,0,0]
    if(len(relative_list_in_fnc)==0):
        return list_x
    if(len(relative_list_in_fnc)==1):
        list_x[6]=relative_list_in_fnc[0][0]
        list_y[6]=relative_list_in_fnc[0][1]

        # filling 0 values

        list_x[5]= list_x[6] * 6 / 7
        list_y[5]= list_y[6] * 6 / 7

        list_x[4]=list_x[6] * 5 / 7
        list_y[4]=list_y[6] * 5 / 7

        list_x[3]=list_x[6] * 4 / 7
        list_y[3]=list_y[6] * 4 / 7

        list_x[2]=list_x[6] * 3 / 7
        list_y[2]=list_y[6] * 3 / 7

        list_x[1]=list_x[6] * 2 / 7
        list_y[1]=list_y[6] * 2 / 7

        list_x[0]=list_x[6] * 1 / 7
        list_y[0]=list_y[6] * 1 / 7
    
    if(len(relative_list_in_fnc)==2):
        list_x[6]=relative_list_in_fnc[1][0]
        list_y[6]=relative_list_in_fnc[1][1]

        list_x[3]=relative_list_in_fnc[0][0]
        list_y[3]=relative_list_in_fnc[0][1]

        # filling 0 values
        # The only case with if:

        # list_x[5]=abs((list_x[6] - list_x[3])) + (list_x[6] - list_x[3]) * 2 / 3 
        # list_y[5]=abs((list_y[6] - list_y[3])) + (list_y[6] - list_y[3]) * 2 / 3 

        # list_x[4]=abs((list_x[6] - list_x[3])) + (list_x[6] - list_x[3]) * 1 / 3
        # list_y[4]=abs((list_y[6] - list_y[3])) + (list_y[6] - list_y[3]) * 1 / 3 

        coords_x = linear_approx_x(list_x[6],list_x[3])
        coords_y = linear_approx_x(list_y[6],list_y[3])

        list_x[5]= coords_x[0]
        list_y[5]= coords_y[0]

        list_x[4]= coords_x[1]
        list_y[4]= coords_y[1]

        list_x[2]=(list_x[3]) * 3 / 4
        list_y[2]=(list_y[3]) * 3 / 4

        list_x[1]=(list_x[3]) * 2 / 4
        list_y[1]=(list_y[3]) * 2 / 4

        list_x[0]=(list_x[3]) * 1 / 4
        list_y[0]=(list_y[3]) * 1 / 4
    
    if(len(relative_list_in_fnc)==3):
        list_x[6]=relative_list_in_fnc[2][0]
        list_y[6]=relative_list_in_fnc[2][1]

        list_x[4]=relative_list_in_fnc[1][0]
        list_y[4]=relative_list_in_fnc[1][1]

        list_x[2]=relative_list_in_fnc[0][0]
        list_y[2]=relative_list_in_fnc[0][1]

        # filling 0 values

        list_x[5]=(list_x[6] + list_x[4]) / 2
        list_y[5]=(list_y[6] + list_y[4]) / 2

        list_x[3]=(list_x[4] + list_x[2]) / 2
        list_y[3]=(list_y[4] + list_y[2]) / 2

        list_x[1]=(list_x[2]) * 2 / 3
        list_y[1]=(list_y[2]) * 2 / 3

        list_x[0]=(list_x[2]) * 1 / 3
        list_y[0]=(list_y[2]) * 1 / 3
    
    if(len(relative_list_in_fnc)==4):
        list_x[6]=relative_list_in_fnc[3][0]
        list_y[6]=relative_list_in_fnc[3][1]

        list_x[4]=relative_list_in_fnc[2][0]
        list_y[4]=relative_list_in_fnc[2][1]

        list_x[3]=relative_list_in_fnc[1][0]
        list_y[3]=relative_list_in_fnc[1][1]

        list_x[1]=relative_list_in_fnc[0][0]
        list_y[1]=relative_list_in_fnc[0][1]

        # filling 0 values

        list_x[5]=(list_x[6] + list_x[4]) / 2
        list_y[5]=(list_y[6] + list_y[4]) / 2

        list_x[2]=(list_x[3] + list_x[1]) / 2
        list_y[2]=(list_y[3] + list_y[1]) / 2

        list_x[0]=(list_x[1]) / 2
        list_y[0]=(list_y[1]) / 2

    if(len(relative_list_in_fnc)==5):
        list_x[6]=relative_list_in_fnc[4][0]
        list_y[6]=relative_list_in_fnc[4][1]

        list_x[4]=relative_list_in_fnc[3][0]
        list_y[4]=relative_list_in_fnc[3][1]

        list_x[3]=relative_list_in_fnc[2][0]
        list_y[3]=relative_list_in_fnc[2][1]

        list_x[1]=relative_list_in_fnc[1][0]
        list_y[1]=relative_list_in_fnc[1][1]

        list_x[0]=relative_list_in_fnc[0][0]
        list_y[0]=relative_list_in_fnc[0][1]

        # filling 0 values

        list_x[5]=(list_x[6] + list_x[4]) / 2
        list_y[5]=(list_y[6] + list_y[4]) / 2

        list_x[2]=(list_x[3] + list_x[1]) / 2
        list_y[2]=(list_y[3] + list_y[1]) / 2
    
    if(len(relative_list_in_fnc)==6):
        list_x[6]=relative_list_in_fnc[5][0]
        list_y[6]=relative_list_in_fnc[5][1]

        list_x[5]=relative_list_in_fnc[4][0]
        list_y[5]=relative_list_in_fnc[4][1]

        list_x[4]=relative_list_in_fnc[3][0]
        list_y[4]=relative_list_in_fnc[3][1]

        list_x[3]=relative_list_in_fnc[2][0]
        list_y[3]=relative_list_in_fnc[2][1]

        list_x[1]=relative_list_in_fnc[1][0]
        list_y[1]=relative_list_in_fnc[1][1]

        list_x[0]=relative_list_in_fnc[0][0]
        list_y[0]=relative_list_in_fnc[0][1]

        # filling 0 values

        list_x[2]=(list_x[3] + list_x[1]) / 2
        list_y[2]=(list_y[3] + list_y[1]) / 2
    
    if(len(relative_list_in_fnc)==7):
        list_x[6]=relative_list_in_fnc[6][0]
        list_y[6]=relative_list_in_fnc[6][1]
        
        list_x[5]=relative_list_in_fnc[5][0]
        list_y[5]=relative_list_in_fnc[5][1]

        list_x[4]=relative_list_in_fnc[4][0]
        list_y[4]=relative_list_in_fnc[4][1]

        list_x[3]=relative_list_in_fnc[3][0]
        list_y[3]=relative_list_in_fnc[3][1]

        list_x[2]=relative_list_in_fnc[2][0]
        list_y[2]=relative_list_in_fnc[2][1]

        list_x[1]=relative_list_in_fnc[1][0]
        list_y[1]=relative_list_in_fnc[1][1]

        list_x[0]=relative_list_in_fnc[0][0]
        list_y[0]=relative_list_in_fnc[0][1]

    return list_x,list_y

In [14]:
# function to store 0's in coordinates list on good places. Every list, no matter how many frames read has to have the same length. 0 will not be read anyway.
# indexes to store how we will pick frames according to how many are they
def define_indexing(length):
    indexes_7 = [ [6],
                [3,6],
                [2,4,6],
                [1,3,4,6],
                [0,1,3,4,6],
                [0,1,3,4,5,6],
                [0,1,2,3,4,5,6]]
    indexes_6 = [ [5],
                [3,5],
                [1,3,5],
                [1,2,4,5],
                [0,1,3,4,5],
                [0,1,2,3,4,5]]
    indexes_5 = [ [4],
                [2,4],
                [1,2,4],
                [0,1,3,4],
                [0,1,2,3,4]]
    indexes_4 = [ [3],
                [1,3],
                [0,1,3],
                [0,1,2,3]]
    indexes_3 = [ [2],
                [1,2],
                [0,1,2]]
    indexes_2 = [ [1],
                [0,1]]
    indexes_1 = [ [0]]

    if(length==1):
        return indexes_1
    if(length==2):
        return indexes_2
    if(length==3):
        return indexes_3
    if(length==4):
        return indexes_4
    if(length==5):
        return indexes_5
    if(length==6):
        return indexes_6
    if(length==7):
        return indexes_7



In [16]:
def relative_change(x_1,y_1,x_2,y_2,distance):
    if(x_2==999 or y_2==999):
        return 0,0
    new_x = (x_1-x_2)/distance
    new_y = (y_1-y_2)/distance
    
    return new_x,new_y
    

In [84]:
#Land mark collection
root_dir="C:\\Users\\User\\Desktop\\moving_frames_left"
letters=os.listdir(root_dir)
letters.sort()
label=-1

beginning_coordinates= []
coordinates_with_movement = []
relative_landmark_list = []
distance = 1

for letter in letters:
    label=label+1
    letter_dir =os.path.join(root_dir,letter)

    numbers=os.listdir(letter_dir)
    # print(numbers)

    for number in numbers:
        input_dir = os.path.join(letter_dir,number)

        images=load_images_from_folder(input_dir)
        os.chdir("C:\\Users\\User\\Desktop\\Sign_language\\PolishAlphabet_SignLanguage_Detection")

        pre_processed_landmark_list = []
        coordinates_with_movement = []
        imageCount =0
        for image in images:
            with mp_hands.Hands(
                static_image_mode=True,
                max_num_hands=2,
                min_detection_confidence=0.7) as hands:

                results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))

                index = 0

                ###############################
                # RIGHT HAND DETECTION
                if (results.multi_hand_landmarks is not None):
                    for idx,hand in enumerate(results.multi_handedness):
                        #print(hand.classification[0].label)
                        if(str(hand.classification[0].label)=="Left"):
                            # print(hand.classification[0].label)
                            # increase only if hand is detected
                            imageCount = imageCount + 1
                            index = idx
                            break
                ###############################
                # print(index)

                if (results.multi_hand_landmarks is not None):
                    # Landmark calculation
                    landmark_list = calc_landmark_list(image, results.multi_hand_landmarks[index])


                    # Conversion to relative coordinates / normalized coordinates
                    if(imageCount==1):
                        pre_processed_landmark_list = pre_process_landmark(
                                landmark_list)
                        beginning_coordinates= [landmark_list[12][0],landmark_list[12][1]]

                        # distance for relative change in movement
                        distance = int(math.sqrt(math.pow(landmark_list[0][0]-landmark_list[5][0],2)+math.pow(landmark_list[0][1]-landmark_list[5][1],2)))
                        if(distance==0):
                            distance=1
                    
                    if(imageCount>1):
                        coordinates_with_movement.append(relative_change(beginning_coordinates[0],beginning_coordinates[1],landmark_list[12][0],landmark_list[12][1],distance))
                    # print(pre_processed_landmark_list)
                    # print(imageCount)
        # after all images are processed

        count = 1
        # print("Fill coords:")
        indexes = define_indexing(len(coordinates_with_movement))

        relative_to_write = []
        # print(coordinates_with_movement)
        if indexes is not None:
            for id in indexes:
                new_list = [coordinates_with_movement[k] for k in id]
                relative_to_write = fill_coordinates(new_list)
                #print(relative_to_write)
                pre_processed_landmark_list.append(relative_to_write[0][0])
                pre_processed_landmark_list.append(relative_to_write[1][0])
                pre_processed_landmark_list.append(relative_to_write[0][1])
                pre_processed_landmark_list.append(relative_to_write[1][1])
                pre_processed_landmark_list.append(relative_to_write[0][2])
                pre_processed_landmark_list.append(relative_to_write[1][2])
                pre_processed_landmark_list.append(relative_to_write[0][3])
                pre_processed_landmark_list.append(relative_to_write[1][3])
                pre_processed_landmark_list.append(relative_to_write[0][4])
                pre_processed_landmark_list.append(relative_to_write[1][4])
                pre_processed_landmark_list.append(relative_to_write[0][5])
                pre_processed_landmark_list.append(relative_to_write[1][5])
                pre_processed_landmark_list.append(relative_to_write[0][6])
                pre_processed_landmark_list.append(relative_to_write[1][6])

                # if(len(pre_processed_landmark_list)!=56):
                #     print("lol")
                # print(len(pre_processed_landmark_list))
                # Write to the dataset file
                logging_csv(label, 1, pre_processed_landmark_list,[])

                # delete last coordinates from list
                pre_processed_landmark_list = pre_processed_landmark_list[:len(pre_processed_landmark_list) - 14]



                            

In [43]:
crds = [(0.03571428571428571, -0.9285714285714286), (0.07142857142857142, -1.2857142857142858), (0.07142857142857142, -1.5357142857142858), (0.0, -1.6428571428571428), (-0.35714285714285715, -5.464285714285714), (-0.21428571428571427, 0.0), (-0.14285714285714285, -0.2857142857142857)]
idx_tmp = [ [6],
            [3,6],
            [2,4,6],
            [1,3,4,6],
            [0,1,3,4,6],
            [0,1,3,4,5,6],
            [0,1,2,3,4,5,6]]

for i in idx_tmp:
    print(type(crds))
    
    new_list = [crds[k] for k in [1,3,4,6]]
    type(new_list)
    print(new_list)

# dodaj zamiane tych zer na cos pomiedzy [0,0.1] - zeby sie nie zjebalo
# augmentacja

<class 'list'>
[(0.07142857142857142, -1.2857142857142858), (0.0, -1.6428571428571428), (-0.35714285714285715, -5.464285714285714), (-0.14285714285714285, -0.2857142857142857)]
<class 'list'>
[(0.07142857142857142, -1.2857142857142858), (0.0, -1.6428571428571428), (-0.35714285714285715, -5.464285714285714), (-0.14285714285714285, -0.2857142857142857)]
<class 'list'>
[(0.07142857142857142, -1.2857142857142858), (0.0, -1.6428571428571428), (-0.35714285714285715, -5.464285714285714), (-0.14285714285714285, -0.2857142857142857)]
<class 'list'>
[(0.07142857142857142, -1.2857142857142858), (0.0, -1.6428571428571428), (-0.35714285714285715, -5.464285714285714), (-0.14285714285714285, -0.2857142857142857)]
<class 'list'>
[(0.07142857142857142, -1.2857142857142858), (0.0, -1.6428571428571428), (-0.35714285714285715, -5.464285714285714), (-0.14285714285714285, -0.2857142857142857)]
<class 'list'>
[(0.07142857142857142, -1.2857142857142858), (0.0, -1.6428571428571428), (-0.35714285714285715, -5.4

In [ ]:
list = [[0.9998931884765625, 2], [0.9999110698699951, 2], [0.9999247789382935, 2], [0.9999165534973145, 2], [0.9999086856842041, 2]]

In [ ]:
list = [[0.9998931884765625, 2], [0.9999110698699951, 2], [0.9999247789382935, 2], [0.9999165534973145, 2], [0.9999086856842041, 2]]